In [ ]:
!pip install edge-tts

  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=27a50fd6421f4a3a6cfde4a08f09d30f09a5993395b0d85fe62c24bf6776db1b
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt


In [ ]:
!pip install moviepy requests

In [ ]:
!mkdir -p /content/working_dir
!mkdir -p /content/final_videos

In [ ]:
from google.colab import userdata
import os
import edge_tts
import requests
from moviepy.editor import VideoFileClip, AudioFileClip
import random
import time


  if event.key is 'enter':



In [ ]:
CLE_API_PIXABAY = userdata.get('cle_api_pixabay')

In [ ]:
deep_thougths = [
    {"text": "Failure is not the end, but a pause before a greater leap. It’s through falling that you learn to rise even higher. Every setback is fuel for your next victory.",
     "prompt": "A tree growing strong despite broken branches, or flowers blooming in a rocky area."},

    {"text": "Just as the ocean pulls back before crashing forward with a wave, your setbacks are the momentum you need to surge ahead. Trust that every retreat leads to a stronger push forward.",
     "prompt": "Waves retreating from the shore before surging forward powerfully."},
]

In [ ]:
def text_to_speech(text:str,voice_id:str,audio_id:str):
  save_path_file = f"/content/working_dir/audio_tts_{audio_id}.mp3"
  communicate = edge_tts.Communicate(text, voice_id)
  communicate.save_sync(save_path_file)

  return save_path_file

def download_video_pixabay(url,file_path):
  response = requests.get(url)
  if response.status_code == 200:
    with open(f"{file_path}", "wb") as f:
      f.write(response.content)
  else:
    raise Exception("Error downloading video")

In [ ]:
#numéro du prompt pour générer la vidéo
i = 1

In [ ]:
text = deep_thougths[i]["text"]
#voice_id = "en-GB-SoniaNeural"
voice_id = "en-US-BrianNeural"
audio_path = text_to_speech(text,voice_id,i)
audio_clip = AudioFileClip(audio_path)
# Calculate the duration to trim the video
audio_duration = audio_clip.duration
trim_duration = audio_duration + 2  # 2 seconds de marge après la fin
print(f"audio duration(s) : {trim_duration}")


audio duration(s) : 11.94


In [ ]:
# L'URL de l'API Pixabay pour les vidéos
url = "https://pixabay.com/api/videos/"
nb_videos = 10
# Paramètres de la requête (par exemple, chercher des vidéos de 'nature')
params = {
    "key": CLE_API_PIXABAY,
    "q": "",
    "per_page": nb_videos,  # Nombre de résultats
}


params["q"] = deep_thougths[i]["prompt"]
response = requests.get(url, params=params)
response.raise_for_status()  # Vérifier si la requête est réussie



In [ ]:
 data = response.json()
 data

{'total': 7744,
 'totalHits': 500,
 'hits': [{'id': 24216,
   'pageURL': 'https://pixabay.com/videos/id-24216/',
   'type': 'film',
   'tags': 'sea, ocean, wave, beach, blue, nature, pacific, sand, water, seashore, guatemala, shore, aerial, live wallpaper',
   'duration': 31,
   'videos': {'large': {'url': 'https://cdn.pixabay.com/video/2019/06/05/24216-340670744_large.mp4',
     'width': 1920,
     'height': 1080,
     'size': 21883755,
     'thumbnail': 'https://cdn.pixabay.com/video/2019/06/05/24216-340670744_large.jpg'},
    'medium': {'url': 'https://cdn.pixabay.com/video/2019/06/05/24216-340670744_medium.mp4',
     'width': 1280,
     'height': 720,
     'size': 10959447,
     'thumbnail': 'https://cdn.pixabay.com/video/2019/06/05/24216-340670744_medium.jpg'},
    'small': {'url': 'https://cdn.pixabay.com/video/2019/06/05/24216-340670744_small.mp4',
     'width': 960,
     'height': 540,
     'size': 6279587,
     'thumbnail': 'https://cdn.pixabay.com/video/2019/06/05/24216-34067

In [ ]:
is_video_duration_ok = False
for idx in range(nb_videos):
  video_duration = data["hits"][idx]["duration"]
  if video_duration> trim_duration:
    is_video_duration_ok = True
    break

if not is_video_duration_ok:
  print("videos too short")
else:
  video_url = data["hits"][idx]["videos"]["small"]["url"]

  # path pour télécharger la vidéo
  video_filename = '/content/working_dir/video.mp4'
  print("Step : Download video")
  # télécharger la vidéo
  download_video_pixabay(video_url, video_filename)

  # Charger la vidéo en tantque clip
  video_clip = VideoFileClip(video_filename)

  #Ajuster la durée de la vidéo avec la durée de l'audio
  trimmed_video_clip = video_clip.subclip(0, trim_duration)

  # ajouter l'audio
  print("Step : Set audio to video")
  final_clip = trimmed_video_clip.set_audio(audio_clip)
  output_path = f'/content/final_videos/generated_video_{i}.mp4'
  print("Step : Save video")
  # enregistrer la vidéo
  final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac')
#vérifier que la vidéo est bien enregistrée
  if os.path.exists(output_path):
      print(f"Video successfully written to {output_path}")
  else:
      print("Failed to write the video file.")


Step : Download video
Step : Set audio to video
Step : Save video
Moviepy - Building video /content/final_videos/generated_video_1.mp4.
MoviePy - Writing audio in generated_video_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/final_videos/generated_video_1.mp4



Moviepy - Done !
Moviepy - video ready /content/final_videos/generated_video_1.mp4
Video successfully written to /content/final_videos/generated_video_1.mp4
